# Assignment #1
# Part 1: Getting Started ( 50 points )

For this assignment, you will set up your class server. This server will be used for in-class exercises and assignments. The server is a Debian 12 virtual machine hosted on a Proxmox cluster in the WPEB T102 server room. Your container has an IP on a private `10.105.0.0/16` subnet within a software-defined network.



TCP Port 8443 is a reverse proxy that uses OpenID Connect (OIDC). This is often called an identity-aware proxy because it inspects the OIDC claim from an identity provider and performs authorization. Your server's identity-aware reverse proxy will only provide access to your UNR username.

## 1. Firewall Configuration ( 20 Points )
Your server already has the dynamic firewall manager `firewalld` installed. 
I've created a few rules already for your server to access ttyd and Jupyter Lab through your browser:
- Port 7681 - ttyd
- Port 8888 - Jupyter Lab

Do not modify these rules, or else you may lock yourself out of your server.

This section walks you through configuring your system's firewall.

### New zone and policy
Nearly all firewall managers have the concept of a `zone`. A `zone` "defines the trust level for a connection, interface or source address binding. This is a one to many relation, which means that a connection, interface or source can only be part of one zone, but a zone can be used for many network connections, interfaces, and sources."

[firewalld zone documentation](https://firewalld.org/documentation/zone/#:~:text=A%20firewall%20zone%20defines%20the,network%20connections%2C%20interfaces%20and%20sources.)

Look through the documentation for `firewalld` and use the `firewall-cmd` to create the following:
 - A `rule` associated with the `public` zone for HTTP and HTTPS
 - Remove `ssh` from the `public` zone.
 - A new `zone` named `lxd`
   - Add two `services` to the zone, `dhcp` and `dns`.
   - Add forwarding to the `lxd` zone.
   - Change the target of the `lxd` zone from `default` to `ACCEPT`.

Execute the commands below the printf statement in the cell below.

In [1]:
# Write your commands here
sudo firewall-cmd --permanent --zone=public --add-service=http
sudo firewall-cmd --permanent --zone=public --add-service=https
sudo firewall-cmd --permanent --zone=public --remove-service=ssh
sudo firewall-cmd --permanent --new-zone=lxd
sudo firewall-cmd --permanent --zone=lxd --add-service=dhcp
sudo firewall-cmd --permanent --zone=lxd --add-service=dns
sudo firewall-cmd --permanent --zone=lxd --add-forward
sudo firewall-cmd --permanent --zone=lxd --set-target=ACCEPT

sudo firewall-cmd --reload


success
success
success
Error: NAME_CONFLICT: new_zone(): 'lxd'
success
success
success
success
success


### How do you know you did it correctly? Use these commands to check the zone configurations.

In [2]:
sudo firewall-cmd --info-zone public

public
  target: default
  icmp-block-inversion: no
  interfaces: 
  sources: 
  services: dhcpv6-client http https
  ports: 8888/tcp 7681/tcp 22/tcp
  protocols: 
  forward: yes
  masquerade: no
  forward-ports: 
  source-ports: 
  icmp-blocks: 
  rich rules: 


In [3]:
sudo firewall-cmd --info-zone lxd

lxd
  target: ACCEPT
  icmp-block-inversion: no
  interfaces: 
  sources: 
  services: dhcp dns
  ports: 
  protocols: 
  forward: yes
  masquerade: no
  forward-ports: 
  source-ports: 
  icmp-blocks: 
  rich rules: 


How can you use nmap to see if the `http` service rule is functioning properly?
Write your answer in the cell below

By specifing port 80 when scanning the container, to double check that the port is open. If the port is open. then the http service is probably functioning correctly. 

## 2. LXD Configuration ( 10 points )
LXD is a Linux container manager that was released in 2014. We are using containers to enhance the security and availability of
the services hosted on your server. LXD will be the sandbox for services that you run on your server. We will use `snap` to install LXD. `snap` is a package manager that works across many different Linux distributions.

In [5]:
sudo apt install -y snapd

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
snapd is already the newest version (2.57.6-1+b5).
0 upgraded, 0 newly installed, 0 to remove and 3 not upgraded.


Test to see if `snap` is installed.

In [6]:
sudo snap --help

The snap command lets you install, configure, refresh and remove snaps.
Snaps are packages that work across many different Linux distributions,
enabling secure delivery and operation of the latest apps and utilities.

Usage: snap <command> [<options>...]

Commonly used commands can be classified as follows:

           Basics: find, info, install, remove, list, components
          ...more: refresh, revert, switch, disable, enable, create-cohort
          History: changes, tasks, abort, watch
          Daemons: services, start, stop, restart, logs
      Permissions: connections, interface, connect, disconnect
    Configuration: get, set, unset, wait
      App Aliases: alias, aliases, unalias, prefer
          Account: login, logout, whoami
        Snapshots: saved, save, check-snapshot, restore, forget
           Device: model, remodel, reboot, recovery
     Quota Groups: set-quota, remove-quota, quotas, quota
  Validation Sets: validate
        ... Other: warnings, okay, known, ack, v

In [7]:
sudo snap install core lxd

snap "core" is already installed, see 'snap help refresh'


In [8]:
sudo snap logs lxd

2025-02-10T14:03:09-08:00 lxd.activate[6250]: ==> Creating missing snap configuration
2025-02-10T14:03:09-08:00 lxd.activate[6250]: ==> Loading snap configuration
2025-02-10T14:03:09-08:00 lxd.activate[6250]: ==> Checking for socket activation support
2025-02-10T14:03:09-08:00 lxd.activate[6250]: ==> Creating "lxd" user
2025-02-10T14:03:09-08:00 useradd[6486]: new user: name=lxd, UID=995, GID=111, home=/var/snap/lxd/common/lxd, shell=/bin/false, from=none
2025-02-10T14:03:09-08:00 lxd.activate[6250]: ==> Setting LXD socket ownership
2025-02-10T14:03:09-08:00 lxd.activate[6250]: ==> Setting LXD user socket ownership
2025-02-10T14:03:09-08:00 lxd.activate[6250]: ==> LXD never started on this system, no need to start it now
2025-02-10T14:03:09-08:00 systemd[1]: snap.lxd.activate.service: Deactivated successfully.
2025-02-10T14:03:09-08:00 systemd[1]: Finished snap.lxd.activate.service - Service for snap application lxd.activate.


Give yourself sudo permissions for lxd and add snap binaries to path

In [9]:
sudo usermod -aG lxd $USER
export PATH=$PATH:/snap/bin # Add snaps to path for current session
echo 'export PATH=$PATH:/snap/bin' >> ~/.bashrc # Make it persist through reboots

Now go through the `lxd init` flow in a terminal window.
 - Do not enable clustering
 - Configure a new storage pool of type `dir`
 - Do not connect to a MAAS server
 - Create a new local network bridge, `lxdbr0`
 - IPv4 and IPv6 CIDRs shoud be `auto`
 - Do not allow LXD to be accessible over the network
 - Update stale images automatically

- Add the `lxdbr0` interface to the `lxd` firewall zone
- Don't forget to run `sudo firewall-cmd --reload` afterwards

In [10]:
# Write your commands here
sudo firewall-cmd --zone=lxd --add-interface=lxdbr0
sudo firewall-cmd --reload


success
success


## 3. How do you know if LXD works? ( 10 points )
Use the `lxc` command to display information about various LXD components.
 - networking
 - storage
 - profiles
 - containers

Hint: Look at the `lxc --help` output.

In [11]:
# Networking
lxc network list

To start your first container, try: lxc launch ubuntu:24.04
Or for a virtual machine: lxc launch ubuntu:24.04 --vm

+---------+----------+---------+----------------+---------------------------+-------------+---------+---------+
|  NAME   |   TYPE   | MANAGED |      IPV4      |           IPV6            | DESCRIPTION | USED BY |  STATE  |
+---------+----------+---------+----------------+---------------------------+-------------+---------+---------+
| enp6s18 | physical | NO      |                |                           |             | 0       |         |
+---------+----------+---------+----------------+---------------------------+-------------+---------+---------+
| lxdbr0  | bridge   | YES     | 10.222.37.1/24 | fd42:ec94:b4a8:7962::1/64 |             | 1       | CREATED |
+---------+----------+---------+----------------+---------------------------+-------------+---------+---------+


In [12]:
# Storage
lxc storage list 

+---------+--------+------------------------------------------------+-------------+---------+---------+
|  NAME   | DRIVER |                     SOURCE                     | DESCRIPTION | USED BY |  STATE  |
+---------+--------+------------------------------------------------+-------------+---------+---------+
| default | dir    | /var/snap/lxd/common/lxd/storage-pools/default |             | 1       | CREATED |
+---------+--------+------------------------------------------------+-------------+---------+---------+


In [13]:
# Profiles
lxc profile list

+---------+---------------------+---------+
|  NAME   |     DESCRIPTION     | USED BY |
+---------+---------------------+---------+
| default | Default LXD profile | 0       |
+---------+---------------------+---------+


In [14]:
# Containers
lxc list

+------+-------+------+------+------+-----------+
| NAME | STATE | IPV4 | IPV6 | TYPE | SNAPSHOTS |
+------+-------+------+------+------+-----------+


## 3. Create a test container
Take a look at the documentation for the `lxc launch` command. Create a container named `test0` it should be based on `ubuntu:22.04`.

In [ ]:
# Launch command here
lxc launch ubuntu:22.04 test0

Testing `test0`

In [ ]:
lxc exec test0 -- hostname


In [37]:
lxc exec test0 -- ip -brief addr show

lo               UNKNOWN        127.0.0.1/8 ::1/128 
eth0@if12        UP             fd42:ec94:b4a8:7962:216:3eff:feb3:7d41/64 fe80::216:3eff:feb3:7d41/64 


Test communication to the Internet

In [ ]:
lxc exec test0 -- ping -c 1 9.9.9.9